In [1]:
import os
os.chdir('D:/github_project/BioXNet')
os.getcwd()

'D:\\github_project\\BioXNet'

In [2]:
import json
import itertools
import argparse
import collections
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import data_loader.data_loader as module_data
import model.loss as module_loss
import model.metric as module_metric
import model.bioxnet as module_arch
from parse_config import ConfigParser
from trainer import Trainer
from utils import inf_loop, MetricTracker
from base import BaseTrainer

# 读取 .json 文件

In [3]:
# 读取 .json 文件保存为 config 对象
with open('D:\github_project\BioXNet\config\gdsc.json') as f:
    config = json.load(f)

print(type(config))
config

<class 'dict'>


{'name': 'BioXNet_GDSC',
 'n_gpu': 1,
 'pretrain': None,
 'transfer_directly': False,
 'arch': {'type': 'BioXNet',
  'args': {'n_hidden_layers': 5,
   'direction': 'root_to_leaf',
   'dropout_list': [0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
   'sparse': True,
   'add_unk_genes': False,
   'batch_normal': True,
   'use_bias': True,
   'attention': True,
   'sparse_first_layer': True,
   'class_num': None,
   'n_outputs': 6,
   'shuffle_genes': False}},
 'data_loader': {'type': 'BioXNetDataLoader',
  'args': {'data_source_list': ['GDSC'],
   'input_data_order': ['drug_target', 'mutation', 'cnv', 'methylation'],
   'batch_size': 1024,
   'seed': 0,
   'shuffle': True,
   'num_workers': 8,
   'eval_dataset': True,
   'pre_params': None,
   'gene_intersection': True,
   'params': {'data_dir': 'data/',
    'remove_response_outlier': True,
    'mut_binary': True,
    'selected_genes': None,
    'combine_type': 'intersection',
    'use_coding_genes_only': True}}},
 'optimizer': {'type': 'Adam',
  '

# 参数设置和 dataloader 对象初始化

In [4]:
# 将 config 转为 ConfigParser 对象（参考 D:\github_project\BioXNet\parse_config.py 中的注释）
config = ConfigParser(config)

In [5]:
# 创建日志记录器
logger = config.get_logger('train')
logger

<Logger train (DEBUG)>

In [6]:
# fix random seeds for reproducibility
SEED = config['data_loader']['args']['seed']
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [7]:
# setup data_loader instances
logger.info('------------------Setup dataloader--------------------')
config['data_loader']['args']['logger'] = logger
data_loader = config.init_obj('data_loader', module_data)
if config['trainer']['class_weights'] != None:
    get_class_weights = True
else:
    get_class_weights = False
train_data_loader, validate_data_loader, test_data_loader, class_weights = data_loader.get_dataloader(get_class_weights)
reactome_network = data_loader.get_reactome()
features, genes = data_loader.get_features_genes()
logger.info('------------------Finish dataloader-------------------')

------------------Setup dataloader--------------------
Loading ['GDSC'] data
Loading processed data...
The shape of cnv_allgenes_df (634, 15059)
The shape of methylation_allgenes_df (708, 15059)
The shape of mutation_allgenes_df (937, 15059)
The shape of drug_target_allgenes_df (221, 15059)
479 samples and 219 drugs in 135353 records.
Transforming the dataframe to dictionary
Keep the genes that existed in GDSC and TCGA.
The number of genes that existed in all datasets: 14812.


D:\github_project\BioXNet\data_loader\data_loader.py:113: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  self.methylation_allgenes_dict = methylation_allgenes_df.T.to_dict('list')


Loading the split index with seed 0.
108282 samples for train, 13536 samples for valid, 13535 samples for test.

Training size 108282
Validation size 13536
Testing size 13535
Not using the class weights
------------------Finish dataloader-------------------


# 模型构建

In [8]:
logger.info('------------------Setup model-------------------------')
config['arch']['args']['features'] = features
config['arch']['args']['genes'] = genes
config['arch']['args']['reactome_network'] = reactome_network
config['arch']['args']['logger'] = logger

------------------Setup model-------------------------


In [9]:
config['arch']

{'type': 'BioXNet',
 'args': {'n_hidden_layers': 5,
  'direction': 'root_to_leaf',
  'dropout_list': [0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
  'sparse': True,
  'add_unk_genes': False,
  'batch_normal': True,
  'use_bias': True,
  'attention': True,
  'sparse_first_layer': True,
  'class_num': None,
  'n_outputs': 6,
  'shuffle_genes': False,
  'features': MultiIndex([(  'A1BG', 'drug_target'),
              (  'A1BG',    'mutation'),
              (  'A1BG',         'cnv'),
              (  'A1BG', 'methylation'),
              (  'A1CF', 'drug_target'),
              (  'A1CF',    'mutation'),
              (  'A1CF',         'cnv'),
              (  'A1CF', 'methylation'),
              (   'A2M', 'drug_target'),
              (   'A2M',    'mutation'),
              ...
              ('ZYG11B',         'cnv'),
              ('ZYG11B', 'methylation'),
              (   'ZYX', 'drug_target'),
              (   'ZYX',    'mutation'),
              (   'ZYX',         'cnv'),
             

In [10]:
model = config.init_obj('arch', module_arch) # 调用 D:\github_project\BioXNet\model\bioxnet.py 中的 'BioXNet' 类，其中 'BioXNet' 由 config['arch']['type'] 定义
logger.info(model)

get layers from root to leaf
layer 0
pathways 1623
genes 9375
filtered_map (9375, 1623)
filtered_map (14812, 1623)
filtered_map (14812, 1623)
layer 0, # of edges 26669.0
layer 1
pathways 1119
genes 1634
filtered_map (1634, 1119)
filtered_map (1623, 1119)
filtered_map (1623, 1119)
layer 1, # of edges 1634.0
layer 2
pathways 490
genes 1121
filtered_map (1121, 490)
filtered_map (1119, 490)
filtered_map (1119, 490)
layer 2, # of edges 1122.0
layer 3
pathways 159
genes 490
filtered_map (490, 159)
filtered_map (490, 159)
filtered_map (490, 159)
layer 3, # of edges 490.0
layer 4
pathways 28
genes 159
filtered_map (159, 28)
filtered_map (159, 28)
filtered_map (159, 28)
layer 4, # of edges 160.0
layer 5
pathways 1
genes 28
filtered_map (28, 1)
filtered_map (28, 1)
filtered_map (28, 1)
layer 5, # of edges 28.0
original dropout list [0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
HiddenLayer-0: n_genes 14812, n_pathways 1623
layer 0, dropout 0.1
HiddenLayer-1: n_genes 1623, n_pathways 1119
layer 1, dropout 0

```config.init_obj('arch', module_arch)``` 等价于 ```getattr(module_data, module_name)(**module_args)``` 或 ```getattr(module_data, 'BioXNet')(**module_args)```

In [11]:
module_name = config['arch']['type']
module_name

'BioXNet'

In [12]:
module_args = dict(config['arch']['args'])
module_args

{'n_hidden_layers': 5,
 'direction': 'root_to_leaf',
 'dropout_list': [0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
 'sparse': True,
 'add_unk_genes': False,
 'batch_normal': True,
 'use_bias': True,
 'attention': True,
 'sparse_first_layer': True,
 'class_num': None,
 'n_outputs': 6,
 'shuffle_genes': False,
 'features': MultiIndex([(  'A1BG', 'drug_target'),
             (  'A1BG',    'mutation'),
             (  'A1BG',         'cnv'),
             (  'A1BG', 'methylation'),
             (  'A1CF', 'drug_target'),
             (  'A1CF',    'mutation'),
             (  'A1CF',         'cnv'),
             (  'A1CF', 'methylation'),
             (   'A2M', 'drug_target'),
             (   'A2M',    'mutation'),
             ...
             ('ZYG11B',         'cnv'),
             ('ZYG11B', 'methylation'),
             (   'ZYX', 'drug_target'),
             (   'ZYX',    'mutation'),
             (   'ZYX',         'cnv'),
             (   'ZYX', 'methylation'),
             ( 'ZZEF1', 'drug

In [13]:
# getattr(module_data, 'BioXNet')(**module_args)
# getattr(module_data, module_name)(**module_args)

## BioXNet 基本架构

### Diagonal 层

In [14]:
features, genes = data_loader.get_features_genes()
feature_names = {}
n_features = len(features)
print(n_features)
n_genes = len(genes)
print(n_genes)

59248
14812


In [15]:
output_dim=n_genes
input_shape=(None, n_features)
input_dim=input_shape[1]
use_bias=True

In [16]:
n_inputs_per_node = input_dim // output_dim
n_inputs_per_node

4

In [17]:
kernel = nn.Parameter(torch.randn(1, input_dim))
print(kernel.size())
print(kernel)

torch.Size([1, 59248])
Parameter containing:
tensor([[-1.3993, -0.0502, -0.2887,  ..., -0.9479,  1.4178,  1.6453]],
       requires_grad=True)


In [18]:
bias = nn.Parameter(torch.randn(output_dim))
print(bias.size())
print(bias)

torch.Size([14812])
Parameter containing:
tensor([ 1.5543,  0.6520,  1.9197,  ...,  0.9392, -0.8891,  0.2526],
       requires_grad=True)


In [19]:
torch.nn.init.xavier_uniform_(kernel)

Parameter containing:
tensor([[ 0.0004, -0.0007, -0.0042,  ..., -0.0011,  0.0009,  0.0054]],
       requires_grad=True)

In [20]:
x = torch.randn(1024, 59248)

In [21]:
mult = x * kernel
print(mult.size())

torch.Size([1024, 59248])


In [22]:
mult = torch.reshape(mult, (-1, n_inputs_per_node))
print(mult.size())

torch.Size([15167488, 4])


In [23]:
mult = torch.sum(mult, dim=1)
print(mult.size())

torch.Size([15167488])


In [24]:
output = torch.reshape(mult, (-1, output_dim))
print(output.size())

torch.Size([1024, 14812])


In [25]:
output = output + bias
print(output.size())

torch.Size([1024, 14812])


### SparseTF 层

In [26]:
output_dim=n_genes
input_shape=(None, n_features)
input_dim=input_shape[1]
use_bias=True

In [27]:
ones_ratio = float(n_features) / np.prod([n_genes, n_features])
logger.info('ones_ratio random {}'.format(ones_ratio))
mapp = np.random.choice([0, 1], size=[n_features, n_genes], p=[1 - ones_ratio, ones_ratio])

ones_ratio random 6.751282743721307e-05


In [28]:
print(type(mapp))
print(mapp.shape)
mapp

<class 'numpy.ndarray'>
(59248, 14812)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [29]:
kernel = nn.Parameter(torch.randn(input_dim, output_dim))
print(kernel.size())
print(kernel)

torch.Size([59248, 14812])
Parameter containing:
tensor([[-0.5686,  0.5019,  0.5488,  ..., -1.2368,  1.1531,  2.1889],
        [-0.7815, -0.5363, -1.0389,  ...,  0.0509, -4.4471,  0.0622],
        [-0.2128,  0.4267, -0.6753,  ..., -0.7280,  0.7411,  0.7749],
        ...,
        [-1.4214, -0.8804, -0.2914,  ..., -2.2599, -0.7144, -0.0495],
        [-1.8648,  1.2495, -0.7177,  ..., -1.2152, -2.2272, -0.0353],
        [-0.3330,  0.3273, -2.3004,  ...,  0.4100, -0.3844,  0.7991]],
       requires_grad=True)


In [30]:
bias = nn.Parameter(torch.randn(output_dim))
print(bias.size())
print(bias)

torch.Size([14812])
Parameter containing:
tensor([ 0.0574, -0.0829, -0.2198,  ..., -0.5550,  0.4927, -1.7949],
       requires_grad=True)


In [31]:
torch.nn.init.xavier_uniform_(kernel)

Parameter containing:
tensor([[ 0.0066, -0.0052,  0.0066,  ..., -0.0075,  0.0079, -0.0073],
        [ 0.0055,  0.0004, -0.0048,  ...,  0.0015, -0.0043, -0.0057],
        [-0.0043, -0.0039, -0.0078,  ...,  0.0081, -0.0031,  0.0063],
        ...,
        [-0.0024, -0.0051,  0.0077,  ...,  0.0064,  0.0068, -0.0053],
        [-0.0073, -0.0072, -0.0048,  ...,  0.0081, -0.0076,  0.0056],
        [ 0.0028,  0.0072,  0.0059,  ..., -0.0059,  0.0033, -0.0053]],
       requires_grad=True)

In [32]:
inputs = torch.randn(1024, 59248)

In [33]:
map = torch.from_numpy(mapp)
map

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)

In [34]:
with torch.no_grad():
    kernel.mul_(map)
print(kernel.size())
print(kernel)

torch.Size([59248, 14812])
Parameter containing:
tensor([[0., -0., 0.,  ..., -0., 0., -0.],
        [0., 0., -0.,  ..., 0., -0., -0.],
        [-0., -0., -0.,  ..., 0., -0., 0.],
        ...,
        [-0., -0., 0.,  ..., 0., 0., -0.],
        [-0., -0., -0.,  ..., 0., -0., 0.],
        [0., 0., 0.,  ..., -0., 0., -0.]], requires_grad=True)


In [35]:
output = torch.matmul(inputs, kernel)
print(output.size())

torch.Size([1024, 14812])


In [36]:
output = output + bias
print(output.size())

torch.Size([1024, 14812])


### Dense 层

In [37]:
output_dim=n_genes
input_shape=(None, n_features)
input_dim=input_shape[1]
use_bias=True

In [38]:
kernel = nn.Parameter(torch.randn(input_dim, output_dim))
print(kernel.size())
print(kernel)

torch.Size([59248, 14812])
Parameter containing:
tensor([[ 0.0268,  0.2950,  0.5064,  ...,  1.3895, -0.9974,  1.0060],
        [ 0.2654,  1.0748,  0.7590,  ..., -0.6839, -0.5579,  0.6773],
        [ 0.9982,  2.2238, -2.0811,  ...,  1.7643,  0.0862,  0.9317],
        ...,
        [ 0.6990, -1.5453, -1.4856,  ...,  0.1301,  0.0953,  0.6427],
        [ 0.6738,  0.3402,  1.2924,  ..., -0.5219, -0.3970, -0.2259],
        [ 0.8237, -0.1481, -0.7947,  ..., -1.0583, -0.1383, -0.7309]],
       requires_grad=True)


In [39]:
bias = nn.Parameter(torch.randn(output_dim))
print(bias.size())
print(bias)

torch.Size([14812])
Parameter containing:
tensor([-0.4411, -1.9112,  0.8708,  ..., -1.3434,  1.8081, -2.7548],
       requires_grad=True)


In [40]:
torch.nn.init.xavier_uniform_(kernel)

Parameter containing:
tensor([[ 0.0028,  0.0014,  0.0080,  ..., -0.0039,  0.0067,  0.0047],
        [-0.0065,  0.0049, -0.0015,  ...,  0.0088, -0.0053, -0.0081],
        [ 0.0028,  0.0015,  0.0029,  ..., -0.0073,  0.0060, -0.0041],
        ...,
        [-0.0028,  0.0035, -0.0052,  ...,  0.0016, -0.0083,  0.0069],
        [-0.0027, -0.0010,  0.0059,  ...,  0.0043, -0.0061,  0.0002],
        [-0.0027,  0.0010,  0.0029,  ..., -0.0068,  0.0007,  0.0025]],
       requires_grad=True)

In [41]:
x = torch.randn(1024, 59248)

In [42]:
x = torch.mm(x, kernel)
print(x.size())

torch.Size([1024, 14812])


In [43]:
x = x + bias
print(x.size())

torch.Size([1024, 14812])


### BioXNet 层

In [44]:
class Diagonal(nn.Module):
    def __init__(self, output_dim, use_bias=True, input_shape=None):
        super(Diagonal, self).__init__()
        self.output_dim = output_dim # 输出维度，指定层的输出大小
        self.use_bias = use_bias # 是否使用偏置
        input_dim = input_shape[1] # 输入维度
        self.n_inputs_per_node = input_dim // self.output_dim # 以第一层为例，output_dim=14812，input_dim=59248，n_inputs_per_node=4

        # create parameter
        self.kernel = nn.Parameter(torch.randn(1, input_dim)) # 权重参数，通过 torch.randn 随机初始化，形状为 [1, input_dim]，其中 input_dim 是输入数据的特征维度。
        if self.use_bias:
            self.bias = nn.Parameter(torch.randn(output_dim)) # 偏置参数，形状为 [output_dim]，通过 torch.randn 随机初始化
        else:
            self.register_parameter('bias', None) # 如果 use_bias 为 False，则不使用偏置，将其设置为 None
        self.__initializer()

    def __initializer(self):
        torch.nn.init.xavier_uniform_(self.kernel) # 使用 Xavier 初始化方法对权重矩阵进行初始化
        if self.bias is not None:
            torch.nn.init.zeros_(self.bias) # 使用零初始化方法将偏置项 self.bias 初始化为全零

    def forward(self, x):
        # kernel is used to assign a weight to each input, shape: [1, genes*4]
        # x：输入张量，形状为 [batch_size, input_dim]=[batch_size, genes*4]
        mult = x * self.kernel # [batch_size, genes*4], element-wise multiplication
        mult = torch.reshape(mult, (-1, self.n_inputs_per_node)) # [batch_size*genes, 4]
        # mult 张量的形状被重新塑形为一个具有 self.n_inputs_per_node 列的矩阵，而行数由 -1 自动确定以保持张量中的元素数量不变
        # e.g., tensor = torch.randn(6, 4) ---> reshaped_tensor = torch.reshape(tensor, (-1, 3)) ---> print(reshaped_tensor.shape)  # 输出：torch.Size([8, 3])
        
        # for each gene, sum the weighted four input together as the output
        mult = torch.sum(mult, dim=1) # [batch_size*genes]
        output = torch.reshape(mult, (-1, self.output_dim)) # [batch_size, genes]

        if self.use_bias:
            output = output + self.bias

        return output

class SparseTF(nn.Module):
    def __init__(self, output_dim, map=None, use_bias=True, input_shape=None):
        '''Sparse Tensor Factorization layer
        
        Args:
            output_dim (_type_): _description_
            map (np.array, binary): the shape is (input_dim, output_dim) and the value is 1 or 0（map 参数是一个二进制数组，表示了输入维度和输出维度之间的稀疏关系）
            use_bias (bool, optional): _description_. Defaults to True.
            input_shape (_type_, optional): _description_. Defaults to None.
        '''
        super(SparseTF, self).__init__()
        self.output_dim = output_dim
        input_dim = input_shape[1]
        self.use_bias = use_bias
        
        self.register_buffer('map', map) # map 张量注册为模型的缓冲区，这意味着它会被添加到模型的参数列表中，但不会被视为模型的可训练参数

        self.kernel = nn.Parameter(torch.randn(input_dim, output_dim))
        if self.use_bias:
            self.bias = nn.Parameter(torch.randn(output_dim))
        else:
            self.register_parameter('bias', None)
        self.__initializer()

    def __initializer(self):
        torch.nn.init.xavier_uniform_(self.kernel)
        if self.bias is not None:
            torch.nn.init.zeros_(self.bias)

    def forward(self, inputs):
        # shape of inputs: (batch_size, input_dim)
        # 通过 torch.no_grad() 禁止梯度计算，然后将核 kernel 乘以稀疏映射 map
        with torch.no_grad():
            self.kernel.mul_(self.map) # 逐元素相乘
        # map = self.map.to(inputs.device)
        # tt = self.kernel * map # element-wise multiplication, shape (input_dim, output_dim)
        # output = torch.matmul(inputs, tt)
        # 计算输入张量 inputs 与核 kernel 的矩阵乘法，得到输出张量 output
        output = torch.matmul(inputs, self.kernel)
        if self.use_bias:
            output = output + self.bias
        return output
    
class Dense(nn.Module):
    def __init__(self, output_dim, input_shape=None, use_bias=True):
        super(Dense, self).__init__()
        input_dim = input_shape[1]
        self.output_dim = output_dim
        self.use_bias = use_bias

        self.kernel = nn.Parameter(torch.randn(input_dim, output_dim))
        if self.use_bias:
            self.bias = nn.Parameter(torch.randn(output_dim))
        else:
            self.register_parameter('bias', None)
        self.__initializer()

    def __initializer(self):
        torch.nn.init.xavier_uniform_(self.kernel)
        if self.bias is not None:
            torch.nn.init.zeros_(self.bias)

    def forward(self, x):
        x = torch.mm(x, self.kernel) # shape: (batch_size, output_dim) torch.mm 函数，用于执行矩阵乘法操作
        if self.use_bias:
            x = x + self.bias
        return x
    
class BioXNetLayer(nn.Module): # BioXNetLayer 本质还是 SparseTF 层或 Dense 层
    def __init__(self, mapp, dropout, sparse, use_bias, attention, batch_normal, class_num, i):
        super(BioXNetLayer, self).__init__()
        self.attention = attention
        self.batch_normal = batch_normal
        self.class_num = class_num
        self.i = i

        n_genes, n_pathways = mapp.shape
        if sparse:
            mapp = self.df_to_tensor(mapp)
            # 该层接收稀疏的二维张量 mapp 作为输入，并根据给定的参数进行初始化
            # 这一层用于处理稀疏数据，其中 n_pathways 表示输出的维度（输出的通路数），mapp 是稀疏的输入张量，use_bias 控制是否使用偏置项，input_shape 表示输入张量的形状
            self.hidden_layer = SparseTF(n_pathways, mapp, use_bias=use_bias, input_shape=(None, n_genes))
        else:
            # 该层将具有 n_genes 个输入特征和 n_pathways 个输出特征，使用权重矩阵和偏置向量进行线性变换。这一层用于处理稠密数据
            self.hidden_layer = Dense(n_pathways, input_shape=(None, n_genes))
        self.activation = nn.Tanh()
        
        if attention:
            # attention 层将具有 n_genes 个输入特征和 n_pathways 个输出特征
            self.attention_prob_layer = Dense(n_pathways, input_shape=(None, n_genes))
            self.attention_activation = nn.Sigmoid()
        
        # testing
        if self.class_num == 2 or self.class_num is None:
            # binary classification or regression, the output is a single number
            self.decision_layer_single_output = nn.Linear(in_features=n_pathways, out_features=1)
            # if self.class_num == 2:
            #     self.decision_activation = nn.Sigmoid()
        else:
            # multi-class classification, the output is a vector
            self.decision_layer_multi_output = nn.Linear(in_features=n_pathways, out_features=self.class_num)
        if batch_normal:
            self.batchnormal_layer = nn.BatchNorm1d(num_features=n_pathways) # 创建了一个批量归一化层，其输入特征的数量为 n_pathways
        
        self.dropout_layer = nn.Dropout(dropout)

    def forward(self, inputs):
        outcome = self.hidden_layer(inputs)
        if self.batch_normal:
            outcome = self.batchnormal_layer(outcome)
        outcome = self.activation(outcome)
        if self.attention:
            attention_probs = self.attention_activation(self.attention_prob_layer(inputs))
            outcome = outcome * attention_probs
        else:
            attention_probs = None
        outcome = self.dropout_layer(outcome)

        if self.class_num == 2 or self.class_num is None:
            decision_outcome = self.decision_layer_single_output(outcome) # decision_outcome 用于计算每一层 BioXNet layer 的损失函数，最终损失函数是每一层损失函数的加权和
        else:
            decision_outcome = self.decision_layer_multi_output(outcome)

        # if self.class_num == 2:
        #     decision_outcome = self.decision_activation(decision_outcome)
    
        return outcome, decision_outcome, attention_probs

    def df_to_tensor(self, df):
        tensor = torch.from_numpy(df.to_numpy()) # 将 DataFrame 转换为 NumPy 数组，然后使用 torch.from_numpy() 将 NumPy 数组转换为 PyTorch 张量
        tensor = tensor.type(torch.FloatTensor) # 将张量的数据类型转换为浮点数类型
        return tensor

<html>
    <img src="./loss_function.png", width=987, heigth=311>
</html>

In [45]:
ones_ratio = float(n_features) / np.prod([n_genes, n_features])
logger.info('ones_ratio random {}'.format(ones_ratio))
mapp = np.random.choice([0, 1], size=[n_features, n_genes], p=[1 - ones_ratio, ones_ratio])

ones_ratio random 6.751282743721307e-05


In [46]:
mapp
dropout=0.5
sparse=config['arch']['args']['sparse']
use_bias=config['arch']['args']['use_bias'] 
attention=config['arch']['args']['attention'] 
batch_normal=config['arch']['args']['batch_normal'] 
class_num=config['arch']['args']['class_num'] 

In [47]:
n_genes, n_pathways = mapp.shape
print(n_genes)
print(n_pathways)

59248
14812


In [48]:
if sparse:
    mapp = torch.from_numpy(mapp)
    mapp = mapp.type(torch.FloatTensor)
    hidden_layer = SparseTF(n_pathways, mapp, use_bias=use_bias, input_shape=(None, n_genes))
else:
    hidden_layer = Dense(n_pathways, input_shape=(None, n_genes))

In [49]:
activation = nn.Tanh()

In [50]:
if attention:
    attention_prob_layer = Dense(n_pathways, input_shape=(None, n_genes))
    attention_activation = nn.Sigmoid()

In [51]:
if class_num == 2 or class_num is None:
    # binary classification or regression, the output is a single number
    decision_layer_single_output = nn.Linear(in_features=n_pathways, out_features=1)
    # if class_num == 2:
    #     decision_activation = nn.Sigmoid()
else:
    # multi-class classification, the output is a vector
    decision_layer_multi_output = nn.Linear(in_features=n_pathways, out_features=class_num)

In [52]:
if batch_normal:
    batchnormal_layer = nn.BatchNorm1d(num_features=n_pathways) # 创建了一个批量归一化层，其输入特征的数量为 n_pathways

In [53]:
dropout_layer = nn.Dropout(dropout)

In [54]:
inputs = torch.randn(1024, 59248)

In [55]:
outcome = hidden_layer(inputs)
print(outcome.size())
print(kernel.data.shape)
print(kernel.data)

torch.Size([1024, 14812])
torch.Size([59248, 14812])
tensor([[ 0.0028,  0.0014,  0.0080,  ..., -0.0039,  0.0067,  0.0047],
        [-0.0065,  0.0049, -0.0015,  ...,  0.0088, -0.0053, -0.0081],
        [ 0.0028,  0.0015,  0.0029,  ..., -0.0073,  0.0060, -0.0041],
        ...,
        [-0.0028,  0.0035, -0.0052,  ...,  0.0016, -0.0083,  0.0069],
        [-0.0027, -0.0010,  0.0059,  ...,  0.0043, -0.0061,  0.0002],
        [-0.0027,  0.0010,  0.0029,  ..., -0.0068,  0.0007,  0.0025]])


In [56]:
outcome = batchnormal_layer(outcome)
print(outcome.size())

torch.Size([1024, 14812])


In [57]:
outcome = activation(outcome)
print(outcome.size())

torch.Size([1024, 14812])


In [58]:
attention_probs = attention_activation(attention_prob_layer(inputs))
print(attention_probs.size())

torch.Size([1024, 14812])


In [59]:
outcome = outcome * attention_probs
print(outcome.size())

torch.Size([1024, 14812])


In [60]:
outcome = dropout_layer(outcome)
print(outcome.size())
print(kernel.data.shape)
print(kernel.data)

torch.Size([1024, 14812])
torch.Size([59248, 14812])
tensor([[ 0.0028,  0.0014,  0.0080,  ..., -0.0039,  0.0067,  0.0047],
        [-0.0065,  0.0049, -0.0015,  ...,  0.0088, -0.0053, -0.0081],
        [ 0.0028,  0.0015,  0.0029,  ..., -0.0073,  0.0060, -0.0041],
        ...,
        [-0.0028,  0.0035, -0.0052,  ...,  0.0016, -0.0083,  0.0069],
        [-0.0027, -0.0010,  0.0059,  ...,  0.0043, -0.0061,  0.0002],
        [-0.0027,  0.0010,  0.0029,  ..., -0.0068,  0.0007,  0.0025]])


In [61]:
if class_num == 2 or class_num is None:
    decision_outcome = decision_layer_single_output(outcome)
else:
    decision_outcome = decision_layer_multi_output(outcome)
print(decision_outcome.size())

torch.Size([1024, 1])


### get_layer_maps

#### 分步实现 

In [62]:
features, genes = data_loader.get_features_genes()
n_hidden_layers=config['arch']['args']['n_hidden_layers']
direction=config['arch']['args']['direction']
add_unk_genes=config['arch']['args']['add_unk_genes']

n_levels=n_hidden_layers # BioXNet 的隐藏层层数对应 reactome network 的 levels
reactome_layers = reactome_network.get_layers(n_levels, direction)
reactome_layers # 储存 reactome network layer 的 list，第一个元素为 root ---> high-order pathways，中间元素为 high-order pathways ---> low-order pathways，最后一个元素为 low-order pathways ---> genes

get layers from root to leaf


[{'root': ['R-HSA-109582',
   'R-HSA-112316',
   'R-HSA-1266738',
   'R-HSA-1430728',
   'R-HSA-1474165',
   'R-HSA-1474244',
   'R-HSA-1500931',
   'R-HSA-162582',
   'R-HSA-1640170',
   'R-HSA-1643685',
   'R-HSA-168256',
   'R-HSA-1852241',
   'R-HSA-382551',
   'R-HSA-392499',
   'R-HSA-397014',
   'R-HSA-400253',
   'R-HSA-4839726',
   'R-HSA-5357801',
   'R-HSA-5653656',
   'R-HSA-69306',
   'R-HSA-73894',
   'R-HSA-74160',
   'R-HSA-8953854',
   'R-HSA-8953897',
   'R-HSA-8963743',
   'R-HSA-9609507',
   'R-HSA-9612973',
   'R-HSA-9709957']},
 {'R-HSA-1500931': ['R-HSA-373753', 'R-HSA-391160', 'R-HSA-446728'],
  'R-HSA-400253': ['R-HSA-1368071', 'R-HSA-1368082', 'R-HSA-1368108'],
  'R-HSA-1474165': ['R-HSA-1187000', 'R-HSA-1500620'],
  'R-HSA-5653656': ['R-HSA-199991', 'R-HSA-2173782'],
  'R-HSA-69306': ['R-HSA-69002', 'R-HSA-69239'],
  'R-HSA-74160': ['R-HSA-211000',
   'R-HSA-212165',
   'R-HSA-73857',
   'R-HSA-73864',
   'R-HSA-74158',
   'R-HSA-75944'],
  'R-HSA-1266738': [

In [63]:
filtering_index = genes # 储存 input feature 包含的所有基因
filtering_index

['A1BG',
 'A1CF',
 'A2M',
 'A2ML1',
 'A4GALT',
 'A4GNT',
 'AAAS',
 'AACS',
 'AADAC',
 'AADACL2',
 'AADACL3',
 'AADACL4',
 'AADAT',
 'AAGAB',
 'AAK1',
 'AAMP',
 'AANAT',
 'AARS2',
 'AASDH',
 'AASDHPPT',
 'AASS',
 'AATF',
 'AATK',
 'ABAT',
 'ABCA1',
 'ABCA12',
 'ABCA13',
 'ABCA2',
 'ABCA3',
 'ABCA4',
 'ABCA5',
 'ABCA6',
 'ABCA7',
 'ABCA8',
 'ABCA9',
 'ABCB1',
 'ABCB10',
 'ABCB11',
 'ABCB4',
 'ABCB5',
 'ABCB6',
 'ABCB8',
 'ABCB9',
 'ABCC1',
 'ABCC10',
 'ABCC11',
 'ABCC12',
 'ABCC2',
 'ABCC3',
 'ABCC4',
 'ABCC5',
 'ABCC6',
 'ABCC8',
 'ABCC9',
 'ABCD2',
 'ABCD3',
 'ABCD4',
 'ABCE1',
 'ABCF1',
 'ABCF2',
 'ABCF3',
 'ABCG1',
 'ABCG2',
 'ABCG4',
 'ABCG5',
 'ABCG8',
 'ABHD1',
 'ABHD10',
 'ABHD11',
 'ABHD12',
 'ABHD12B',
 'ABHD13',
 'ABHD14A',
 'ABHD14B',
 'ABHD15',
 'ABHD2',
 'ABHD3',
 'ABHD4',
 'ABHD5',
 'ABHD6',
 'ABHD8',
 'ABI1',
 'ABI2',
 'ABI3',
 'ABI3BP',
 'ABL1',
 'ABL2',
 'ABLIM1',
 'ABLIM2',
 'ABLIM3',
 'ABO',
 'ABR',
 'ABRA',
 'ABT1',
 'ABTB1',
 'ABTB2',
 'ACAA1',
 'ACAA2',
 'ACACA',
 

In [64]:
maps = []

In [65]:
print(len(reactome_layers[::-1]))
reactome_layers[::-1] # 颠倒 reactome_layers 的顺序，第一个元素为 genes ---> low-order pathways，中间元素为 low-order pathways ---> high-order pathways，最后一个元素为 high-order pathways ---> root

6


[{'R-HSA-110362': array(['APEX1', 'FEN1', 'LIG1', 'PARG', 'PARP1', 'PARP2', 'POLB'],
        dtype=object),
  'R-HSA-5651801': array(['FEN1', 'LIG1', 'PCNA', 'POLB', 'POLD1', 'POLD2', 'POLD3', 'POLD4',
         'POLE', 'POLE2', 'POLE3', 'POLE4', 'RFC1', 'RFC2', 'RFC3', 'RFC4',
         'RFC5', 'RPA1', 'RPA2', 'RPA3'], dtype=object),
  'R-HSA-73930': array(['POLB'], dtype=object),
  'R-HSA-111461': array(['APIP', 'AVEN', 'CARD8', 'CASP3', 'CASP7', 'CASP9', 'CYCS',
         'DIABLO', 'MAPK1', 'MAPK3', 'UACA', 'XIAP'], dtype=object),
  'R-HSA-264870': array(['CASP3', 'CASP6', 'CASP7', 'CASP8', 'DBNL', 'GAS2', 'GSN', 'MAPT',
         'PLEC', 'SPTAN1', 'VIM'], dtype=object),
  'R-HSA-351906': array(['CDH1', 'CTNNB1', 'DSG1', 'DSG2', 'DSG3', 'DSP', 'OCLN', 'PKP1',
         'TJP1', 'TJP2'], dtype=object),
  'R-HSA-352238': array(['LMNA', 'LMNB1'], dtype=object),
  'R-HSA-111469': array(['CASP3', 'CASP7', 'CASP9', 'CYCS', 'DIABLO', 'SEPTIN4', 'XIAP'],
        dtype=object),
  'R-HSA-111457': a

In [66]:
enum_list = list(enumerate(reactome_layers[::-1])) # 枚举 reactome_layers[::-1] list，返回索引和对应元素值
enum_list[0] # 第一个元素为 genes ---> low-order pathways

(0,
 {'R-HSA-110362': array(['APEX1', 'FEN1', 'LIG1', 'PARG', 'PARP1', 'PARP2', 'POLB'],
        dtype=object),
  'R-HSA-5651801': array(['FEN1', 'LIG1', 'PCNA', 'POLB', 'POLD1', 'POLD2', 'POLD3', 'POLD4',
         'POLE', 'POLE2', 'POLE3', 'POLE4', 'RFC1', 'RFC2', 'RFC3', 'RFC4',
         'RFC5', 'RPA1', 'RPA2', 'RPA3'], dtype=object),
  'R-HSA-73930': array(['POLB'], dtype=object),
  'R-HSA-111461': array(['APIP', 'AVEN', 'CARD8', 'CASP3', 'CASP7', 'CASP9', 'CYCS',
         'DIABLO', 'MAPK1', 'MAPK3', 'UACA', 'XIAP'], dtype=object),
  'R-HSA-264870': array(['CASP3', 'CASP6', 'CASP7', 'CASP8', 'DBNL', 'GAS2', 'GSN', 'MAPT',
         'PLEC', 'SPTAN1', 'VIM'], dtype=object),
  'R-HSA-351906': array(['CDH1', 'CTNNB1', 'DSG1', 'DSG2', 'DSG3', 'DSP', 'OCLN', 'PKP1',
         'TJP1', 'TJP2'], dtype=object),
  'R-HSA-352238': array(['LMNA', 'LMNB1'], dtype=object),
  'R-HSA-111469': array(['CASP3', 'CASP7', 'CASP9', 'CYCS', 'DIABLO', 'SEPTIN4', 'XIAP'],
        dtype=object),
  'R-HSA-111457

In [67]:
i, layer = enum_list[0]
print(i)
print(layer)
print(len(layer))

0
{'R-HSA-110362': array(['APEX1', 'FEN1', 'LIG1', 'PARG', 'PARP1', 'PARP2', 'POLB'],
      dtype=object), 'R-HSA-5651801': array(['FEN1', 'LIG1', 'PCNA', 'POLB', 'POLD1', 'POLD2', 'POLD3', 'POLD4',
       'POLE', 'POLE2', 'POLE3', 'POLE4', 'RFC1', 'RFC2', 'RFC3', 'RFC4',
       'RFC5', 'RPA1', 'RPA2', 'RPA3'], dtype=object), 'R-HSA-73930': array(['POLB'], dtype=object), 'R-HSA-111461': array(['APIP', 'AVEN', 'CARD8', 'CASP3', 'CASP7', 'CASP9', 'CYCS',
       'DIABLO', 'MAPK1', 'MAPK3', 'UACA', 'XIAP'], dtype=object), 'R-HSA-264870': array(['CASP3', 'CASP6', 'CASP7', 'CASP8', 'DBNL', 'GAS2', 'GSN', 'MAPT',
       'PLEC', 'SPTAN1', 'VIM'], dtype=object), 'R-HSA-351906': array(['CDH1', 'CTNNB1', 'DSG1', 'DSG2', 'DSG3', 'DSP', 'OCLN', 'PKP1',
       'TJP1', 'TJP2'], dtype=object), 'R-HSA-352238': array(['LMNA', 'LMNB1'], dtype=object), 'R-HSA-111469': array(['CASP3', 'CASP7', 'CASP9', 'CYCS', 'DIABLO', 'SEPTIN4', 'XIAP'],
      dtype=object), 'R-HSA-111457': array(['BAX', 'CYCS', 'DIABLO'

##### get_map_from_layer

###### 分步实现

In [68]:
layer_dict = layer
pathways = list(layer_dict.keys())
print('pathways', len(pathways))
print(pathways)

pathways 1623
['R-HSA-110362', 'R-HSA-5651801', 'R-HSA-73930', 'R-HSA-111461', 'R-HSA-264870', 'R-HSA-351906', 'R-HSA-352238', 'R-HSA-111469', 'R-HSA-111457', 'R-HSA-111885', 'R-HSA-111933', 'R-HSA-112399', 'R-HSA-110056', 'R-HSA-112411', 'R-HSA-113510', 'R-HSA-111446', 'R-HSA-111447', 'R-HSA-111448', 'R-HSA-139910', 'R-HSA-139915', 'R-HSA-1169091', 'R-HSA-1169092', 'R-HSA-2025928', 'R-HSA-1169408', 'R-HSA-8983711', 'R-HSA-1222387', 'R-HSA-1222449', 'R-HSA-1222538', 'R-HSA-1222541', 'R-HSA-1236973', 'R-HSA-1236974', 'R-HSA-1236977', 'R-HSA-1236978', 'R-HSA-1296041', 'R-HSA-141333', 'R-HSA-141334', 'R-HSA-3371378', 'R-HSA-69416', 'R-HSA-141430', 'R-HSA-141444', 'R-HSA-1461957', 'R-HSA-1462054', 'R-HSA-1482788', 'R-HSA-1482798', 'R-HSA-1482801', 'R-HSA-1482839', 'R-HSA-1482883', 'R-HSA-1482922', 'R-HSA-1482925', 'R-HSA-1483076', 'R-HSA-1483101', 'R-HSA-1483115', 'R-HSA-1483148', 'R-HSA-1483152', 'R-HSA-1483166', 'R-HSA-1483171', 'R-HSA-1483191', 'R-HSA-1483196', 'R-HSA-1483213', 'R-HSA-1

In [69]:
genes = list(itertools.chain.from_iterable(layer_dict.values()))
print('genes', len(genes))
print(genes)

genes 33265
['APEX1', 'FEN1', 'LIG1', 'PARG', 'PARP1', 'PARP2', 'POLB', 'FEN1', 'LIG1', 'PCNA', 'POLB', 'POLD1', 'POLD2', 'POLD3', 'POLD4', 'POLE', 'POLE2', 'POLE3', 'POLE4', 'RFC1', 'RFC2', 'RFC3', 'RFC4', 'RFC5', 'RPA1', 'RPA2', 'RPA3', 'POLB', 'APIP', 'AVEN', 'CARD8', 'CASP3', 'CASP7', 'CASP9', 'CYCS', 'DIABLO', 'MAPK1', 'MAPK3', 'UACA', 'XIAP', 'CASP3', 'CASP6', 'CASP7', 'CASP8', 'DBNL', 'GAS2', 'GSN', 'MAPT', 'PLEC', 'SPTAN1', 'VIM', 'CDH1', 'CTNNB1', 'DSG1', 'DSG2', 'DSG3', 'DSP', 'OCLN', 'PKP1', 'TJP1', 'TJP2', 'LMNA', 'LMNB1', 'CASP3', 'CASP7', 'CASP9', 'CYCS', 'DIABLO', 'SEPTIN4', 'XIAP', 'BAX', 'CYCS', 'DIABLO', 'GSDMD', 'GSDME', 'SEPTIN4', 'ADCY2', 'ADCY3', 'ADCY4', 'ADCY5', 'ADCY6', 'ADCY7', 'ADCY8', 'ADCY9', 'AHCYL1', 'CALM1', 'CAMK2A', 'CAMK2B', 'CAMK2D', 'CAMK2G', 'CAMK4', 'CAMKK1', 'CAMKK2', 'CDK5', 'CREB1', 'GNA11', 'GNA14', 'GNA15', 'GNAI1', 'GNAI2', 'GNAI3', 'GNAL', 'GNAQ', 'GNAT3', 'GNB1', 'GNB2', 'GNB3', 'GNB4', 'GNB5', 'GNG10', 'GNG11', 'GNG12', 'GNG13', 'GNG2', '

In [70]:
genes = list(np.unique(genes))
print('genes', len(genes))
print(genes)

genes 9375
['28S rRNA', '45S pre-rRNA gene', '5.8S rRNA', '5S rRNA', '7SL RNA (ENSG00000222619)', '7SL RNA (ENSG00000222639)', '7a', '8b', '9b', 'A2M', 'AASS', 'ABCA1', 'ABCA12', 'ABCA13', 'ABCA2', 'ABCA3', 'ABCA5', 'ABCA6', 'ABCA7', 'ABCA9', 'ABCB4', 'ABCB6', 'ABCB7', 'ABCB8', 'ABCC2', 'ABCC3', 'ABCC4', 'ABCC9', 'ABCD1', 'ABCD2', 'ABCD3', 'ABCD4', 'ABCG1', 'ABCG2', 'ABCG4', 'ABCG5', 'ABCG8', 'ABHD17B', 'ABHD17C', 'ABHD6', 'ABI2', 'ABL1', 'ABL2', 'ABLIM2', 'ABLIM3', 'ABR', 'ACAA1', 'ACACB', 'ACADM', 'ACADS', 'ACADSB', 'ACADVL', 'ACAT1', 'ACBD5', 'ACE2', 'ACER2', 'ACER3', 'ACHE', 'ACKR2', 'ACKR3', 'ACKR4', 'ACLY', 'ACMSD', 'ACOT4', 'ACOT6', 'ACOT8', 'ACOX1', 'ACOX2', 'ACOX3', 'ACOXL', 'ACP3', 'ACSBG2', 'ACSF3', 'ACSL1', 'ACSL3', 'ACSL4', 'ACSL5', 'ACSL6', 'ACSM2A', 'ACSM2B', 'ACSM3', 'ACSM4', 'ACSM5', 'ACSM6', 'ACSS2', 'ACSS3', 'ACTB', 'ACTC1', 'ACTG1', 'ACTG2', 'ACTL6A', 'ACTL6B', 'ACTN1', 'ACTN2', 'ACTN3', 'ACTN4', 'ACTR10', 'ACTR1A', 'ACTR1B', 'ACTR2', 'ACTR3', 'ACTR5', 'ACTR8', 'ACV

In [71]:
pathways.sort()
print(pathways)
genes.sort()
print(genes)

print(len(pathways)) # 共 1623 条 pathways
print(len(genes)) # 共 9375 个基因

['R-HSA-1059683', 'R-HSA-110056', 'R-HSA-110312', 'R-HSA-110314', 'R-HSA-110320', 'R-HSA-110328', 'R-HSA-110329', 'R-HSA-110330', 'R-HSA-110331', 'R-HSA-110357', 'R-HSA-110362', 'R-HSA-111367', 'R-HSA-111446', 'R-HSA-111447', 'R-HSA-111448', 'R-HSA-111452', 'R-HSA-111453', 'R-HSA-111457', 'R-HSA-111461', 'R-HSA-111469', 'R-HSA-111885', 'R-HSA-111933', 'R-HSA-112122', 'R-HSA-112126', 'R-HSA-112303', 'R-HSA-112308', 'R-HSA-112382', 'R-HSA-112399', 'R-HSA-112411', 'R-HSA-113418', 'R-HSA-113510', 'R-HSA-114294', 'R-HSA-114516', 'R-HSA-114604', 'R-HSA-114608', 'R-HSA-1169091', 'R-HSA-1169092', 'R-HSA-1169408', 'R-HSA-1170546', 'R-HSA-1221632', 'R-HSA-1222387', 'R-HSA-1222449', 'R-HSA-1222538', 'R-HSA-1222541', 'R-HSA-1234158', 'R-HSA-1234176', 'R-HSA-1236382', 'R-HSA-1236973', 'R-HSA-1236974', 'R-HSA-1236977', 'R-HSA-1236978', 'R-HSA-1237044', 'R-HSA-1237112', 'R-HSA-1247673', 'R-HSA-1250196', 'R-HSA-1250342', 'R-HSA-1250347', 'R-HSA-1251932', 'R-HSA-1251985', 'R-HSA-1253288', 'R-HSA-126669

In [72]:
mat = np.zeros((len(pathways), len(genes)))

In [73]:
items_list = list(layer_dict.items()) # 将 layer_dict 转换为一个键值对的列表，其中每个元素是一个元组，包含字典中的键和对应的值
items_list[0] # 第一个元素包含 'R-HSA-110362' pathway 及其对应的基因

('R-HSA-110362',
 array(['APEX1', 'FEN1', 'LIG1', 'PARG', 'PARP1', 'PARP2', 'POLB'],
       dtype=object))

In [74]:
p, gs = items_list[0]
print(p)
print(gs)

R-HSA-110362
['APEX1' 'FEN1' 'LIG1' 'PARG' 'PARP1' 'PARP2' 'POLB']


In [75]:
g_inds = [genes.index(g) for g in gs]
print(g_inds) # layer_dict 第一个元素包含的基因在 genes 中的索引
p_ind = pathways.index(p) # layer_dict 第一个元素包含的 pathways 在 pathways 中的索引
print(p_ind)

[367, 2637, 4329, 5766, 5771, 5775, 6191]
10


In [76]:
mat[p_ind, g_inds] = 1
np.savetxt('./coding_practice/mat_R-HSA-110362.csv', mat, delimiter=',')

In [77]:
mat = np.zeros((len(pathways), len(genes)))
for p, gs in layer_dict.items():
    g_inds = [genes.index(g) for g in gs]
    p_ind = pathways.index(p)
    mat[p_ind, g_inds] = 1
np.savetxt('./coding_practice/mat.csv', mat, delimiter=',')

###### 调用函数

In [78]:
def get_map_from_layer(layer_dict):
    pathways = list(layer_dict.keys())
    print('pathways', len(pathways))
    genes = list(itertools.chain.from_iterable(layer_dict.values()))
    genes = list(np.unique(genes))
    print('genes', len(genes))
    pathways.sort()
    genes.sort()

    n_pathways = len(pathways)
    n_genes = len(genes)

    mat = np.zeros((n_pathways, n_genes))
    for p, gs in layer_dict.items():
        g_inds = [genes.index(g) for g in gs]
        p_ind = pathways.index(p)
        mat[p_ind, g_inds] = 1

    df = pd.DataFrame(mat, index=pathways, columns=genes)

    return df.T

In [79]:
mapp = get_map_from_layer(layer)
mapp

pathways 1623
genes 9375


,R-HSA-1059683,R-HSA-110056,R-HSA-110312,R-HSA-110314,R-HSA-110320,R-HSA-110328,R-HSA-110329,R-HSA-110330,R-HSA-110331,R-HSA-110357,...,R-HSA-977068,R-HSA-977225,R-HSA-977347,R-HSA-977444,R-HSA-977606,R-HSA-982772,R-HSA-983168,R-HSA-983170,R-HSA-983189,R-HSA-983695
28S rRNA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45S pre-rRNA gene,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.8S rRNA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5S rRNA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7SL RNA (ENSG00000222619),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trxA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
trxB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
vif,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
vpr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
filter_df = pd.DataFrame(index=filtering_index)
filter_df

""
A1BG
A1CF
A2M
A2ML1
A4GALT
...
ZXDC
ZYG11A
ZYG11B
ZYX


In [81]:
filtered_map = filter_df.merge(mapp, right_index=True, left_index=True, how='left')
# how='left'：表示使用 filter_df 的索引为基础进行合并，保留 filter_df 的所有行，如果在 mapp 中找不到对应的列，则用 NaN 填充
print(filtered_map.shape)
filtered_map

(14812, 1623)


,R-HSA-1059683,R-HSA-110056,R-HSA-110312,R-HSA-110314,R-HSA-110320,R-HSA-110328,R-HSA-110329,R-HSA-110330,R-HSA-110331,R-HSA-110357,...,R-HSA-977068,R-HSA-977225,R-HSA-977347,R-HSA-977444,R-HSA-977606,R-HSA-982772,R-HSA-983168,R-HSA-983170,R-HSA-983189,R-HSA-983695
A1BG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A1CF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A2M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2ML1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A4GALT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZYG11A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZYG11B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZYX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
if add_unk_genes:
    logger.info('Add UNK')
    filtered_map['UNK'] = 0
    ind = filtered_map.sum(axis=1) == 0
    filtered_map.loc[ind, 'UNK'] = 1

In [83]:
filtered_map = filtered_map.fillna(0)
print(filtered_map.shape)
filtered_map

(14812, 1623)


,R-HSA-1059683,R-HSA-110056,R-HSA-110312,R-HSA-110314,R-HSA-110320,R-HSA-110328,R-HSA-110329,R-HSA-110330,R-HSA-110331,R-HSA-110357,...,R-HSA-977068,R-HSA-977225,R-HSA-977347,R-HSA-977444,R-HSA-977606,R-HSA-982772,R-HSA-983168,R-HSA-983170,R-HSA-983189,R-HSA-983695
A1BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1CF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2ML1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A4GALT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZYG11A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZYG11B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZYX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
filtering_index = filtered_map.columns
print('layer {}, # of edges {}'.format(i, filtered_map.sum().sum()))
filtering_index

layer 0, # of edges 26669.0


Index(['R-HSA-1059683', 'R-HSA-110056', 'R-HSA-110312', 'R-HSA-110314',
       'R-HSA-110320', 'R-HSA-110328', 'R-HSA-110329', 'R-HSA-110330',
       'R-HSA-110331', 'R-HSA-110357',
       ...
       'R-HSA-977068', 'R-HSA-977225', 'R-HSA-977347', 'R-HSA-977444',
       'R-HSA-977606', 'R-HSA-982772', 'R-HSA-983168', 'R-HSA-983170',
       'R-HSA-983189', 'R-HSA-983695'],
      dtype='object', length=1623)

##### for i, layer in enumerate(reactome_layers[::-1]): 

In [85]:
features, genes = data_loader.get_features_genes()
filtering_index = genes # 储存 input feature 包含的所有基因
filtering_index

['A1BG',
 'A1CF',
 'A2M',
 'A2ML1',
 'A4GALT',
 'A4GNT',
 'AAAS',
 'AACS',
 'AADAC',
 'AADACL2',
 'AADACL3',
 'AADACL4',
 'AADAT',
 'AAGAB',
 'AAK1',
 'AAMP',
 'AANAT',
 'AARS2',
 'AASDH',
 'AASDHPPT',
 'AASS',
 'AATF',
 'AATK',
 'ABAT',
 'ABCA1',
 'ABCA12',
 'ABCA13',
 'ABCA2',
 'ABCA3',
 'ABCA4',
 'ABCA5',
 'ABCA6',
 'ABCA7',
 'ABCA8',
 'ABCA9',
 'ABCB1',
 'ABCB10',
 'ABCB11',
 'ABCB4',
 'ABCB5',
 'ABCB6',
 'ABCB8',
 'ABCB9',
 'ABCC1',
 'ABCC10',
 'ABCC11',
 'ABCC12',
 'ABCC2',
 'ABCC3',
 'ABCC4',
 'ABCC5',
 'ABCC6',
 'ABCC8',
 'ABCC9',
 'ABCD2',
 'ABCD3',
 'ABCD4',
 'ABCE1',
 'ABCF1',
 'ABCF2',
 'ABCF3',
 'ABCG1',
 'ABCG2',
 'ABCG4',
 'ABCG5',
 'ABCG8',
 'ABHD1',
 'ABHD10',
 'ABHD11',
 'ABHD12',
 'ABHD12B',
 'ABHD13',
 'ABHD14A',
 'ABHD14B',
 'ABHD15',
 'ABHD2',
 'ABHD3',
 'ABHD4',
 'ABHD5',
 'ABHD6',
 'ABHD8',
 'ABI1',
 'ABI2',
 'ABI3',
 'ABI3BP',
 'ABL1',
 'ABL2',
 'ABLIM1',
 'ABLIM2',
 'ABLIM3',
 'ABO',
 'ABR',
 'ABRA',
 'ABT1',
 'ABTB1',
 'ABTB2',
 'ACAA1',
 'ACAA2',
 'ACACA',
 

In [86]:
for i, layer in enumerate(reactome_layers[::-1]):
    logger.info(f'layer {i}')
    # mapp (pd.DataFrame): the columns are pathways and the rows are genes
    mapp = get_map_from_layer(layer)
    # print the position of the nonzero elements in the first row
    # print(mapp.iloc[0, :].to_numpy().nonzero())
    filter_df = pd.DataFrame(index=filtering_index)
    logger.info(f'filtered_map {mapp.shape}')
    # only keep the genes that existed in the input data
    filtered_map = filter_df.merge(mapp, right_index=True, left_index=True, how='left')
    
    logger.info(f'filtered_map {filtered_map.shape}')

    # UNK, add a node for genes without known reactome annotation
    if add_unk_genes:
        logger.info('Add UNK')
        filtered_map['UNK'] = 0
        ind = filtered_map.sum(axis=1) == 0
        filtered_map.loc[ind, 'UNK'] = 1

    filtered_map = filtered_map.fillna(0)
    logger.info(f'filtered_map {filtered_map.shape}')
    # filtering_index = list(filtered_map.columns)
    filtering_index = filtered_map.columns
    logger.info('layer {}, # of edges {}'.format(i, filtered_map.sum().sum()))
    maps.append(filtered_map)

layer 0
pathways 1623
genes 9375
filtered_map (9375, 1623)
filtered_map (14812, 1623)
filtered_map (14812, 1623)
layer 0, # of edges 26669.0
layer 1
pathways 1119
genes 1634
filtered_map (1634, 1119)
filtered_map (1623, 1119)
filtered_map (1623, 1119)
layer 1, # of edges 1634.0
layer 2
pathways 490
genes 1121
filtered_map (1121, 490)
filtered_map (1119, 490)
filtered_map (1119, 490)
layer 2, # of edges 1122.0
layer 3
pathways 159
genes 490
filtered_map (490, 159)
filtered_map (490, 159)
filtered_map (490, 159)
layer 3, # of edges 490.0
layer 4
pathways 28
genes 159
filtered_map (159, 28)
filtered_map (159, 28)
filtered_map (159, 28)
layer 4, # of edges 160.0
layer 5
pathways 1
genes 28
filtered_map (28, 1)
filtered_map (28, 1)
filtered_map (28, 1)
layer 5, # of edges 28.0


In [87]:
maps[0]

,R-HSA-1059683,R-HSA-110056,R-HSA-110312,R-HSA-110314,R-HSA-110320,R-HSA-110328,R-HSA-110329,R-HSA-110330,R-HSA-110331,R-HSA-110357,...,R-HSA-977068,R-HSA-977225,R-HSA-977347,R-HSA-977444,R-HSA-977606,R-HSA-982772,R-HSA-983168,R-HSA-983170,R-HSA-983189,R-HSA-983695
A1BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1CF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2ML1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A4GALT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZYG11A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZYG11B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZYX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
maps[1]

,R-HSA-110312,R-HSA-110314,R-HSA-110320,R-HSA-110357,R-HSA-110373,R-HSA-110381,R-HSA-111367,R-HSA-111452,R-HSA-111453,R-HSA-111465,...,R-HSA-975957,R-HSA-977225,R-HSA-977347,R-HSA-977443,R-HSA-977606,R-HSA-982772,R-HSA-983168,R-HSA-983170,R-HSA-983189,R-HSA-983695
R-HSA-1059683,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-110056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-110312,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-110314,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-110320,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R-HSA-982772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
R-HSA-983168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
R-HSA-983170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
R-HSA-983189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [89]:
maps[2]

,R-HSA-109606,R-HSA-110313,R-HSA-110314,R-HSA-111367,R-HSA-112303,R-HSA-112308,R-HSA-112310,R-HSA-112311,R-HSA-112313,R-HSA-112314,...,R-HSA-9750126,R-HSA-975634,R-HSA-975956,R-HSA-975957,R-HSA-977225,R-HSA-977347,R-HSA-982772,R-HSA-983169,R-HSA-983189,R-HSA-983705
R-HSA-110312,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-110314,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-110320,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-110357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-110373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R-HSA-982772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
R-HSA-983168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
R-HSA-983170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
R-HSA-983189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [90]:
maps[3]

,R-HSA-109581,R-HSA-112307,R-HSA-112315,R-HSA-1181150,R-HSA-1187000,R-HSA-1268020,R-HSA-1280215,R-HSA-1280218,R-HSA-1296071,R-HSA-1362409,...,R-HSA-9675132,R-HSA-9675135,R-HSA-9675143,R-HSA-9675151,R-HSA-9690406,R-HSA-9716542,R-HSA-9717189,R-HSA-977225,R-HSA-983231,R-HSA-983712
R-HSA-109606,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-110313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-110314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-111367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-112303,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R-HSA-977347,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-982772,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-983169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-983189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [91]:
maps[4]

,R-HSA-109582,R-HSA-112316,R-HSA-1266738,R-HSA-1430728,R-HSA-1474165,R-HSA-1474244,R-HSA-1500931,R-HSA-162582,R-HSA-1640170,R-HSA-1643685,...,R-HSA-5653656,R-HSA-69306,R-HSA-73894,R-HSA-74160,R-HSA-8953854,R-HSA-8953897,R-HSA-8963743,R-HSA-9609507,R-HSA-9612973,R-HSA-9709957
R-HSA-109581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-112307,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-112315,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-1181150,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-1187000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R-HSA-9716542,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-9717189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
R-HSA-977225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R-HSA-983231,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
maps[5]

,root
R-HSA-109582,1.0
R-HSA-112316,1.0
R-HSA-1266738,1.0
R-HSA-1430728,1.0
R-HSA-1474165,1.0
R-HSA-1474244,1.0
R-HSA-1500931,1.0
R-HSA-162582,1.0
R-HSA-1640170,1.0
R-HSA-1643685,1.0


#### 调用函数

In [93]:
def get_layer_maps(genes, n_levels, direction, add_unk_genes):
    # reactome_layers is a list and each element is a dataframe
    # the order of elements in reactome_layers is from the source to the terminal nodes
    reactome_layers = reactome_network.get_layers(n_levels, direction) # list
    filtering_index = genes
    maps = []
    # reactomen_layers[::-1] is the reverse order
    for i, layer in enumerate(reactome_layers[::-1]):
        logger.info(f'layer {i}')
        # mapp (pd.DataFrame): the columns are pathways and the rows are genes
        mapp = get_map_from_layer(layer)
        # print the position of the nonzero elements in the first row
        # print(mapp.iloc[0, :].to_numpy().nonzero())
        filter_df = pd.DataFrame(index=filtering_index)
        logger.info(f'filtered_map {mapp.shape}')
        # only keep the genes that existed in the input data
        filtered_map = filter_df.merge(mapp, right_index=True, left_index=True, how='left')
        
        logger.info(f'filtered_map {filtered_map.shape}')

        # UNK, add a node for genes without known reactome annotation
        if add_unk_genes:
            logger.info('Add UNK')
            filtered_map['UNK'] = 0
            ind = filtered_map.sum(axis=1) == 0
            filtered_map.loc[ind, 'UNK'] = 1

        filtered_map = filtered_map.fillna(0)
        logger.info(f'filtered_map {filtered_map.shape}')
        # filtering_index = list(filtered_map.columns)
        filtering_index = filtered_map.columns
        logger.info('layer {}, # of edges {}'.format(i, filtered_map.sum().sum()))
        maps.append(filtered_map)
    return maps

In [94]:
features, genes = data_loader.get_features_genes()
n_hidden_layers=config['arch']['args']['n_hidden_layers']
direction=config['arch']['args']['direction']
add_unk_genes=config['arch']['args']['add_unk_genes']

n_levels=n_hidden_layers # BioXNet 的隐藏层层数对应 reactome network 的 levels
reactome_layers = reactome_network.get_layers(n_levels, direction)
reactome_layers # 储存 reactome network layer 的 list，第一个元素为 root ---> high-order pathways，中间元素为 high-order pathways ---> low-order pathways，最后一个元素为 low-order pathways ---> genes

get layers from root to leaf


[{'root': ['R-HSA-109582',
   'R-HSA-112316',
   'R-HSA-1266738',
   'R-HSA-1430728',
   'R-HSA-1474165',
   'R-HSA-1474244',
   'R-HSA-1500931',
   'R-HSA-162582',
   'R-HSA-1640170',
   'R-HSA-1643685',
   'R-HSA-168256',
   'R-HSA-1852241',
   'R-HSA-382551',
   'R-HSA-392499',
   'R-HSA-397014',
   'R-HSA-400253',
   'R-HSA-4839726',
   'R-HSA-5357801',
   'R-HSA-5653656',
   'R-HSA-69306',
   'R-HSA-73894',
   'R-HSA-74160',
   'R-HSA-8953854',
   'R-HSA-8953897',
   'R-HSA-8963743',
   'R-HSA-9609507',
   'R-HSA-9612973',
   'R-HSA-9709957']},
 {'R-HSA-1500931': ['R-HSA-373753', 'R-HSA-391160', 'R-HSA-446728'],
  'R-HSA-400253': ['R-HSA-1368071', 'R-HSA-1368082', 'R-HSA-1368108'],
  'R-HSA-1474165': ['R-HSA-1187000', 'R-HSA-1500620'],
  'R-HSA-5653656': ['R-HSA-199991', 'R-HSA-2173782'],
  'R-HSA-69306': ['R-HSA-69002', 'R-HSA-69239'],
  'R-HSA-74160': ['R-HSA-211000',
   'R-HSA-212165',
   'R-HSA-73857',
   'R-HSA-73864',
   'R-HSA-74158',
   'R-HSA-75944'],
  'R-HSA-1266738': [

In [95]:
maps = get_layer_maps(genes=genes, n_levels=n_hidden_layers, direction=direction, add_unk_genes=add_unk_genes)
maps

get layers from root to leaf
layer 0
pathways 1623
genes 9375
filtered_map (9375, 1623)
filtered_map (14812, 1623)
filtered_map (14812, 1623)
layer 0, # of edges 26669.0
layer 1
pathways 1119
genes 1634
filtered_map (1634, 1119)
filtered_map (1623, 1119)
filtered_map (1623, 1119)
layer 1, # of edges 1634.0
layer 2
pathways 490
genes 1121
filtered_map (1121, 490)
filtered_map (1119, 490)
filtered_map (1119, 490)
layer 2, # of edges 1122.0
layer 3
pathways 159
genes 490
filtered_map (490, 159)
filtered_map (490, 159)
filtered_map (490, 159)
layer 3, # of edges 490.0
layer 4
pathways 28
genes 159
filtered_map (159, 28)
filtered_map (159, 28)
filtered_map (159, 28)
layer 4, # of edges 160.0
layer 5
pathways 1
genes 28
filtered_map (28, 1)
filtered_map (28, 1)
filtered_map (28, 1)
layer 5, # of edges 28.0


[        R-HSA-1059683  R-HSA-110056  R-HSA-110312  R-HSA-110314  R-HSA-110320  \
 A1BG              0.0           0.0           0.0           0.0           0.0   
 A1CF              0.0           0.0           0.0           0.0           0.0   
 A2M               0.0           0.0           0.0           0.0           0.0   
 A2ML1             0.0           0.0           0.0           0.0           0.0   
 A4GALT            0.0           0.0           0.0           0.0           0.0   
 ...               ...           ...           ...           ...           ...   
 ZXDC              0.0           0.0           0.0           0.0           0.0   
 ZYG11A            0.0           0.0           0.0           0.0           0.0   
 ZYG11B            0.0           0.0           0.0           0.0           0.0   
 ZYX               0.0           0.0           0.0           0.0           0.0   
 ZZEF1             0.0           0.0           0.0           0.0           0.0   
 
         R-HSA

## 分步构建 BioXNet（实现 class BioXNet）

In [96]:
features, genes = data_loader.get_features_genes()
reactome_network = data_loader.get_reactome()
n_hidden_layers=config['arch']['args']['n_hidden_layers']
direction=config['arch']['args']['direction']
dropout_list=config['arch']['args']['dropout_list']
sparse=config['arch']['args']['sparse']
add_unk_genes=config['arch']['args']['add_unk_genes']
batch_normal=config['arch']['args']['batch_normal']
class_num=config['arch']['args']['class_num']
n_outputs=config['arch']['args']['n_outputs']
use_bias=config['arch']['args']['use_bias']
shuffle_genes=config['arch']['args']['shuffle_genes']
attention=config['arch']['args']['attention']
sparse_first_layer=config['arch']['args']['sparse_first_layer']

In [97]:
if sparse:
    # the difference between SparseTF and Diagonal is that the kernel in SparseTF is in shape (n_features, n_genes)
    # while the kernel in Diagonal is in shape (1, n_features)
    if shuffle_genes == 'all':
        # create mapp randomly to shuffle the genes
        ones_ratio = float(n_features) / np.prod([n_genes, n_features])
        logger.info('ones_ratio random {}'.format(ones_ratio))
        # shape of mapp: (n_features, n_genes)
        mapp = np.random.choice([0, 1], size=[n_features, n_genes], p=[1 - ones_ratio, ones_ratio])
        layer1 = SparseTF(output_dim=n_genes, map=mapp, use_bias=use_bias, input_shape=(None, n_features))
    else:
        layer1 = Diagonal(output_dim=n_genes, input_shape=(None, n_features), use_bias=use_bias)
else:
    if sparse_first_layer:
        layer1 = Diagonal(output_dim=n_genes, input_shape=(None, n_features), use_bias=use_bias)
    else:
        layer1 = Dense(output_dim=n_genes, input_shape=(None, n_features), use_bias=use_bias)

In [98]:
layer1_activation = nn.Tanh()
if attention:
    attention_prob_layer = Diagonal(n_genes, input_shape=(None, n_features))
    attention_activation = nn.Sigmoid() 
dropout1 = nn.Dropout(dropout_list[0])

In [99]:
# testing
if class_num == 2 or class_num is None:
    # binary classification or regression, the output is a single number
    decision_layer1_single_output = nn.Linear(in_features=n_genes, out_features=1)
    # if class_num == 2:
    #     decision_activation1 = nn.Sigmoid()
else:
    # multi-class classification, the output is a vector
    decision_layer1_multi_output = nn.Linear(in_features=n_genes, out_features=class_num)
batchnorm_layer1 = nn.BatchNorm1d(num_features=n_genes)

In [100]:
def shuffle_genes_map(self, mapp):
    self.logger.info('shuffling')
    ones_ratio = np.sum(mapp) / np.prod(mapp.shape)
    self.logger.info('ones_ratio {}'.format(ones_ratio))
    mapp = np.random.choice([0, 1], size=mapp.shape, p=[1 - ones_ratio, ones_ratio])
    self.logger.info('random map ones_ratio {}'.format(ones_ratio))
    return mapp

In [101]:
module_list = nn.ModuleList()
if n_hidden_layers > 0:            
    maps = get_layer_maps(genes=genes, n_levels=n_hidden_layers, direction=direction, add_unk_genes=add_unk_genes)
    logger.info(f'original dropout list {dropout_list}')
    dropouts = dropout_list[1:]
    # no considering the last map (layer)
    for i, mapp in enumerate(maps[0: -1]):
        # mapp (pd.DataFrame): the rows are genes and the columns are pathways
        dropout = dropouts[i]
        names = mapp.index
        if shuffle_genes in ['all', 'pathways']:
            mapp = shuffle_genes_map(mapp)
        n_genes, n_pathways = mapp.shape
        logger.info('HiddenLayer-{}: n_genes {}, n_pathways {}'.format(i, n_genes, n_pathways))
        logger.info('layer {}, dropout {}'.format(i, dropout))
        pnet_layer = BioXNetLayer(mapp=mapp, 
                                    dropout=dropout, 
                                    sparse=sparse, 
                                    use_bias=use_bias, 
                                    attention=attention, 
                                    batch_normal=batch_normal,
                                    class_num=class_num,
                                    i=i)
        module_list.add_module('HiddenLayer{}'.format(i), pnet_layer)
        feature_names['h{}'.format(i)] = names
    i = len(maps)
    feature_names['h{}'.format(i-1)] = maps[-1].index
module_list = module_list

get layers from root to leaf
layer 0
pathways 1623
genes 9375
filtered_map (9375, 1623)
filtered_map (14812, 1623)
filtered_map (14812, 1623)
layer 0, # of edges 26669.0
layer 1
pathways 1119
genes 1634
filtered_map (1634, 1119)
filtered_map (1623, 1119)
filtered_map (1623, 1119)
layer 1, # of edges 1634.0
layer 2
pathways 490
genes 1121
filtered_map (1121, 490)
filtered_map (1119, 490)
filtered_map (1119, 490)
layer 2, # of edges 1122.0
layer 3
pathways 159
genes 490
filtered_map (490, 159)
filtered_map (490, 159)
filtered_map (490, 159)
layer 3, # of edges 490.0
layer 4
pathways 28
genes 159
filtered_map (159, 28)
filtered_map (159, 28)
filtered_map (159, 28)
layer 4, # of edges 160.0
layer 5
pathways 1
genes 28
filtered_map (28, 1)
filtered_map (28, 1)
filtered_map (28, 1)
layer 5, # of edges 28.0
original dropout list [0.5, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
HiddenLayer-0: n_genes 14812, n_pathways 1623
layer 0, dropout 0.1
HiddenLayer-1: n_genes 1623, n_pathways 1119
layer 1, dropout 0

In [102]:
# output layer
if class_num == 2 or class_num is None:
    # binary classification or regression, the output is a single number
    output_layer_single_output = nn.Sequential(
        nn.Linear(in_features=n_pathways, out_features=n_pathways),
        nn.LeakyReLU(),
        nn.Linear(in_features=n_pathways, out_features=1)
    )
else:
    # multi-class classification, the output is a vector
    output_layer_multi_output = nn.Sequential(
        nn.Linear(in_features=n_pathways, out_features=n_pathways),
        nn.LeakyReLU(),
        nn.Linear(in_features=n_pathways, out_features=class_num)
    )

# 模型训练

In [103]:
if config['pretrain'] is not None:
    logger.info('Loading the pre-trained model from {} ...'.format(config['pretrain']))
    checkpoint = torch.load(config['pretrain'])
    pretrained_dict = checkpoint['state_dict']
    model_dict = model.state_dict()
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)
else:
    logger.info('Training the model from scratch ...')

if config['transfer_directly'] == True:
    logger.info('Transfer the model directly, only train prediction layer.')
    for name, param in model.named_parameters():
        if 'output' not in name:
            param.requires_grad = False

Training the model from scratch ...


In [104]:
# get function handles of loss and metrics
criterion = getattr(module_loss, config['loss'])
criterion

<function model.loss.rmse_loss(output, target)>

In [105]:
metrics = [getattr(module_metric, met) for met in config['metrics']]
metrics

[<function model.metric.rmse(y_pred, y_true)>,
 <function model.metric.r_squared(y_pred, y_true)>,
 <function model.metric.pearson_roi(y_pred, y_true)>,
 <function model.metric.pearson_pval(y_pred, y_true)>]

In [106]:
trainable_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = config.init_obj('optimizer', torch.optim, trainable_params)
lr_scheduler = config.init_obj('lr_scheduler', torch.optim.lr_scheduler, optimizer)
params = sum([np.prod(p.size()) for p in model.parameters() if p.requires_grad])
logger.info(f'Trainable parameters {params}.')
logger.info('------------------Finish setting up model--------------')

Trainable parameters 53183868.
------------------Finish setting up model--------------


In [107]:
model

BioXNet(
  (layer1): Diagonal()
  (layer1_activation): Tanh()
  (attention_prob_layer): Diagonal()
  (attention_activation): Sigmoid()
  (dropout1): Dropout(p=0.5, inplace=False)
  (decision_layer1_single_output): Linear(in_features=14812, out_features=1, bias=True)
  (batchnorm_layer1): BatchNorm1d(14812, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (module_list): ModuleList(
    (0): BioXNetLayer(
      (hidden_layer): SparseTF()
      (activation): Tanh()
      (attention_prob_layer): Dense()
      (attention_activation): Sigmoid()
      (decision_layer_single_output): Linear(in_features=1623, out_features=1, bias=True)
      (batchnormal_layer): BatchNorm1d(1623, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout_layer): Dropout(p=0.1, inplace=False)
    )
    (1): BioXNetLayer(
      (hidden_layer): SparseTF()
      (activation): Tanh()
      (attention_prob_layer): Dense()
      (attention_activation): Sigmoid()
      (decision_lay

In [117]:
model
criterion
metric_fns=metrics
optimizer
config

In [119]:
BaseTrainer = BaseTrainer(model, criterion, metric_fns, optimizer, config)
BaseTrainer

In [120]:
model
criterion
metric_fns=metrics
optimizer
config
train_data_loader
valid_data_loader=validate_data_loader
test_data_loader
lr_scheduler
len_epoch=None
class_weights
class_num=config['arch']['args']['class_num']
n_outputs=config['arch']['args']['n_outputs']
loss_weights=config['trainer']['loss_weights']
prediction_output=config['trainer']['prediction_output']
save_attention=False

In [121]:
if len_epoch is None:
    # epoch-based training
    len_epoch = len(train_data_loader)
else:
    # iteration-based training
    train_data_loader = inf_loop(train_data_loader)
    len_epoch = len_epoch
len_epoch

106

In [122]:
do_validation = valid_data_loader is not None
do_validation

True

In [123]:
log_step = int(np.sqrt(train_data_loader.batch_size))
log_step

32

In [125]:
train_metrics = MetricTracker('loss', *[m.__name__ for m in metric_fns], writer=BaseTrainer.writer)
valid_metrics = MetricTracker('loss', *[m.__name__ for m in metric_fns], writer=BaseTrainer.writer)

In [130]:
'''
    计算损失函数:
        - 如果输出有多个，即 self.n_outputs > 1，则遍历每个输出，并根据权重 self.loss_weights 计算加权损失
            损失的计算通过 self.criterion 完成，如果存在类别权重 self.class_weights，则将其应用于损失计算中
        - 如果只有一个输出，即 self.n_outputs == 1，则直接计算输出和目标之间的损失。同样，如果存在类别权重 self.class_weights，也会将其应用于损失计算中
'''
def _compute_loss(self, outcome, decision_outcomes, target):
    # loss = self._compute_loss(outcome=outcome, decision_outcomes=decision_outcomes, target=target) # 计算损失函数
    if self.n_outputs > 1:
        output = decision_outcomes
        loss = 0
        for i, lw in enumerate(self.loss_weights): # "loss_weights": [2, 7, 20, 54, 148, 400]: why?
            if self.class_weights is None:
                loss += lw * self.criterion(output[i], target)
            else:
                class_weights = torch.tensor(self.class_weights, dtype=torch.float32).to(target.device)
                loss += lw * self.criterion(output[i], target, class_weights)
        return loss
    else:
        output = outcome
        if self.class_weights is None:
            return self.criterion(output, target)
        else:
            class_weights = torch.tensor(self.class_weights, dtype=torch.float32).to(target.device)
            return self.criterion(output, target, class_weights)


'''
    获取预测分数:
        - 如果有多个输出（self.n_outputs > 1），则根据 self.prediction_output 参数来计算预测分数：
            - 如果 self.prediction_output 为 'average'，则计算所有输出的平均值
            - 如果 self.prediction_output 为 'weighted'，则根据损失权重 self.loss_weights 加权计算输出的加权平均值
            - 如果 self.prediction_output 其他值，则取最后一个输出
        - 如果只有一个输出（self.n_outputs == 1），则根据 self.class_num 参数和输出类型进行如下处理：
            - 如果是多类别分类（self.class_num > 2），则对输出进行 softmax 归一化处理
            - 如果是二分类（self.class_num == 2），则对输出进行 sigmoid 处理
'''
def _get_prediction_scores(self, outcome, decision_outcomes):
    if self.n_outputs > 1:
        if self.class_num is not None and self.class_num > 2:
            # multiclass classification
            decision_outcomes = [torch.log_softmax(v, dim=1) for v in decision_outcomes]
        if self.class_num is not None and self.class_num == 2:
            # binary classification
            decision_outcomes = [torch.sigmoid(v) for v in decision_outcomes]

        output = [a.cpu().detach().numpy() for a in decision_outcomes]
        if self.prediction_output == 'average':
            prediction_score = np.mean(output, axis=0)
        elif self.prediction_output == 'weighted':
            prediction_score = np.average(output, axis=0, weights=self.loss_weights)
        else:
            prediction_score = output[-1]
            
    else:
        if self.class_num is not None and self.class_num > 2:
            outcome = torch.log_softmax(outcome, dim=1)
        if self.class_num is not None and self.class_num == 2:
            outcome = torch.sigmoid(outcome)
        prediction_score = outcome.cpu().detach().numpy()
    return prediction_score

def _progress(self, batch_idx):
    base = '[{}/{} ({:.0f}%)]'
    if hasattr(self.train_data_loader, 'n_samples'):
        current = batch_idx * self.train_data_loader.batch_size
        total = self.train_data_loader.n_samples
    else:
        current = batch_idx
        total = self.len_epoch
    return base.format(current, total, 100.0 * current / total)


In [128]:
model.train()

BioXNet(
  (layer1): Diagonal()
  (layer1_activation): Tanh()
  (attention_prob_layer): Diagonal()
  (attention_activation): Sigmoid()
  (dropout1): Dropout(p=0.5, inplace=False)
  (decision_layer1_single_output): Linear(in_features=14812, out_features=1, bias=True)
  (batchnorm_layer1): BatchNorm1d(14812, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (module_list): ModuleList(
    (0): BioXNetLayer(
      (hidden_layer): SparseTF()
      (activation): Tanh()
      (attention_prob_layer): Dense()
      (attention_activation): Sigmoid()
      (decision_layer_single_output): Linear(in_features=1623, out_features=1, bias=True)
      (batchnormal_layer): BatchNorm1d(1623, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout_layer): Dropout(p=0.1, inplace=False)
    )
    (1): BioXNetLayer(
      (hidden_layer): SparseTF()
      (activation): Tanh()
      (attention_prob_layer): Dense()
      (attention_activation): Sigmoid()
      (decision_lay

In [129]:
train_metrics.reset()

In [131]:
# train_data_loader_iter = iter(train_data_loader)
# batch_idx, (_, _, data, target) = next(enumerate(train_data_loader_iter))

In [ ]:
for batch_idx, (_, _, data, target) in enumerate(train_data_loader):
    data, target = data.to(BaseTrainer.device), target.to(BaseTrainer.device)

    optimizer.zero_grad()
    if n_outputs > 1:
        decision_outcomes, _ = model(data)
        outcome = None
    else:
        outcome, _ = model(data)
        decision_outcomes = None

    loss = _compute_loss(outcome=outcome, decision_outcomes=decision_outcomes, target=target) # 计算损失函数
    loss.backward() # 反向传播
    optimizer.step() # optimizer 更新参数

    BaseTrainer.writer.set_step((epoch - 1) * len_epoch + batch_idx)
    train_metrics.update('loss', loss.item()) # 使用 update 方法将新的指标值添加到指定的指标名称下，在训练过程中跟踪和记录各种指标的变化
    with torch.no_grad(): # 不计算梯度，计算预测结果，并更新评估指标
        y_pred = _get_prediction_scores(outcome, decision_outcomes).squeeze()
        y_true = target.cpu().detach().numpy().squeeze()
        for met in metric_fns:
            if met.__name__ not in ['roc_auc', 'multi_precision']:
                train_metrics.update(met.__name__, met(y_pred, y_true))

    if batch_idx % log_step == 0:
        logger.debug('Train Epoch: {} {} Loss: {:.6f}'.format(
            epoch,
            _progress(batch_idx),
            loss.item()))

    if batch_idx == len_epoch:
        break